In [1]:
from IPython.display import display

from io import StringIO
import itertools
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd

from mosek.fusion import Model, Domain, Expr, ObjectiveSense, SolutionStatus


In [2]:
supply = pd.read_csv('20201007_da_co.processed.csv').set_index('id')
nsupply = len(supply)
print(supply.shape)
supply.head()

(3617, 3)


,node,capacity (MW),offer ($/MW)
id,,,
A3153_0,North,1.0,8.16
A3153_1,North,245.0,9.54
A3153_2,North,1.0,14.27
A3153_3,North,72.0,14.28
A3165_0,Central,180.0,10.77


In [3]:
nodes = list(supply['node'].unique())
nnodes = len(nodes)
print(nodes)

['North', 'Central', 'South']


In [4]:
demand = pd.read_csv('20201007_bids_cb.processed.csv').set_index('id')
assert demand.node.isin(nodes).all()
ndemand = len(demand)
print(demand.shape)
demand.head()

(309, 3)


,node,demand (MW),bid ($/MW)
id,,,
122073561,North,6.0,2000.0
767746013,North,14.0,2000.0
122073033,Central,1228.0,2000.0
122073042,North,1644.0,2000.0
450523929,Central,132.4,2000.0


In [5]:
demand.groupby(demand['bid ($/MW)'] >= 2000.)['node'].count()

bid ($/MW)
False     41
True     268
Name: node, dtype: int64

In [6]:
lines = pd.DataFrame([
    ['%s-%s' % (src, dest), src, dest, 5e3, 1e5] 
    for src, dest in itertools.combinations(nodes, 2)
], columns=['id', 'source', 'dest', 'capacity (MW)', 'susceptance (S)']).set_index('id')
nlines = len(lines)
lines

,source,dest,capacity (MW),susceptance (S)
id,,,,
North-Central,North,Central,5000.0,100000.0
North-South,North,South,5000.0,100000.0
Central-South,Central,South,5000.0,100000.0


In [7]:
rev_lines = lines.copy()
rev_lines.source = lines.dest
rev_lines.dest = lines.source
bi_lines = pd.concat([lines, rev_lines]).set_index(['source', 'dest'], drop=False)
n_bi_lines = len(bi_lines)
bi_lines


source     dest  capacity (MW)  susceptance (S)
source  dest                                                     
North   Central    North  Central         5000.0         100000.0
        South      North    South         5000.0         100000.0
Central South    Central    South         5000.0         100000.0
        North    Central    North         5000.0         100000.0
South   North      South    North         5000.0         100000.0
        Central    South  Central         5000.0         100000.0

In [8]:
pd.concat([demand.groupby('node')[['demand (MW)']].sum(),
           supply.groupby('node')[['capacity (MW)']].sum()], axis='columns')

,demand (MW),capacity (MW)
node,,
Central,28181.2,85287.0
North,12854.2,52353.8
South,16425.5,47877.5


In [9]:
M = Model('kkt')

log = StringIO()
M.setLogHandler(log)

## variables

# primal variables
pD = M.variable('pD', ndemand, Domain.inRange(0., demand['demand (MW)'].values))
pG = M.variable('pG', nsupply, Domain.inRange(0., supply['capacity (MW)'].values))
theta = M.variable('theta', nnodes, Domain.inRange(
    [0.] + [-np.pi] * (nnodes - 1), 
    [0.] + [np.pi] * (nnodes - 1), 
))
# (theta reference value imposed as variable bound)

# line flow artificial variable, to simplify the MOSEK API calls
line_flows = M.variable('flows', n_bi_lines, Domain.greaterThan(0.))
src_idx = [nodes.index(x) for x in bi_lines.source]
dst_idx = [nodes.index(x) for x in bi_lines.dest]
phase_diff = Expr.sub(theta.pick(src_idx), theta.pick(dst_idx))
phase_flow = M.constraint(
    Expr.sub(line_flows, Expr.mulElm(bi_lines['susceptance (S)'].values, phase_diff)),
    Domain.equalsTo(0.)
)

#dual variables
muDup = M.variable('muDup', ndemand, Domain.greaterThan(0.))
muDdown = M.variable('muDdown', ndemand, Domain.greaterThan(0.))
muGup = M.variable('muGup', nsupply, Domain.greaterThan(0.))
muGdown = M.variable('muGdown', nsupply, Domain.greaterThan(0.))
etaup = M.variable('etaup', n_bi_lines, Domain.greaterThan(0.))
etadown = M.variable('etadown', n_bi_lines, Domain.greaterThan(0.))
lambdaN = M.variable('lambdaN', nnodes, Domain.unbounded())
gamma = M.variable('gamma', 1, Domain.unbounded())

## first order conditions

#dL/dpD
demand_idx = [nodes.index(x) for x in demand.node]
dldpd = Expr.sub(muDup, demand['bid ($/MW)'].values)
dldpd = Expr.sub(dldpd, muDdown)
dldpd = Expr.add(dldpd, lambdaN.pick(demand_idx))
pd_const = M.constraint('dldpd', dldpd, Domain.greaterThan(0.))

#dL/dpG
supply_idx = [nodes.index(x) for x in supply.node]
dldpg = Expr.add(muGup, supply['offer ($/MW)'].values)
dldpg = Expr.sub(dldpg, muGdown)
dldpg = Expr.sub(dldpg, lambdaN.pick(supply_idx))
pg_const = M.constraint('dldpg', dldpg, Domain.greaterThan(0.))

#dL/dtheta
theta_eqs = []
for n, n_node in enumerate(nodes):
    if n == 0:
        dldtheta = gamma
    else:
        dldtheta = Expr.zeros(1)
    for m_node in bi_lines.xs(n_node).index:
        m = nodes.index(m_node)

        n_m_idx = bi_lines.index.get_loc((n_node, m_node))
        m_n_idx = bi_lines.index.get_loc((m_node, n_node))
        nm_duals = Expr.sub(etaup.index(n_m_idx), 
                            etaup.index(m_n_idx))
        nm_duals = Expr.sub(nm_duals, etadown.index(n_m_idx))
        nm_duals = Expr.add(nm_duals, etadown.index(m_n_idx))
        nm_duals = Expr.add(nm_duals, lambdaN.index(n))
        nm_duals = Expr.sub(nm_duals, lambdaN.index(m))
        dldtheta = Expr.add(
            dldtheta, 
            Expr.mul(bi_lines.loc[(n_node, m_node), 'susceptance (S)'],
                     nm_duals))
    theta_eqs.append(
        M.constraint('dldtheta%d' % n, dldtheta, Domain.equalsTo(0.)))

## primal equality constraints

# node balance equations    
node_balance_eqs = []
for node in nodes:
    supply_idx = np.flatnonzero(supply.node == node).astype('int32')
    node_supply = Expr.sum(pG.pick(supply_idx))
    demand_idx = np.flatnonzero(demand.node == node).astype('int32')
    node_demand = Expr.sum(pD.pick(demand_idx))
    balance = Expr.sub(node_demand, node_supply)
    out_line_idx = np.flatnonzero(bi_lines.source == node).astype('int32')
    balance = Expr.sub(balance, Expr.sum(line_flows.pick(out_line_idx)))
    in_line_idx = np.flatnonzero(bi_lines.dest == node).astype('int32')
    balance = Expr.add(balance, Expr.sum(line_flows.pick(in_line_idx)))
    balance_eq = M.constraint(node + 'balance', balance, Domain.equalsTo(0.))
    node_balance_eqs.append(balance_eq)
    
## bilinearity constraints
bigM = 5e5  # It's M. It's Big. It's Big-M!
binary_domain = Domain.binary()
# binary_domain = Domain.inRange(0., 1.)

#0 ≤ −pd + PD ⊥ μdup ≥ 0 ∀d
z_upper_d = M.variable('z_upper_d', ndemand, binary_domain)
M.constraint(Expr.sub(Expr.sub(demand['demand (MW)'].values, pD),
                      Expr.mul(bigM, z_upper_d)), Domain.lessThan(0.))
not_z_upper_d = Expr.sub(1., z_upper_d)
M.constraint(Expr.sub(muDup,
                      Expr.mul(bigM, not_z_upper_d)), Domain.lessThan(0.))

#0 ≤ pd ⊥ μddown ≥ 0 ∀d
z_lower_d = M.variable('z_lower_d', ndemand, binary_domain)
M.constraint(Expr.sub(pD,
                      Expr.mul(bigM, z_lower_d)), Domain.lessThan(0.))
not_z_lower_d = Expr.sub(1., z_lower_d)
M.constraint(Expr.sub(muDdown,
                      Expr.mul(bigM, not_z_lower_d)), Domain.lessThan(0.))

#0 ≤ −pg + PG ⊥ μgup ≥ 0 ∀g
z_upper_g = M.variable('z_upper_g', nsupply, binary_domain)
M.constraint(Expr.sub(Expr.sub(supply['capacity (MW)'].values, pG),
                      Expr.mul(bigM, z_upper_g)), Domain.lessThan(0.))
not_z_upper_g = Expr.sub(1., z_upper_g)
M.constraint(Expr.sub(muGup,
                      Expr.mul(bigM, not_z_upper_g)), Domain.lessThan(0.))

#0 ≤ pg ⊥ μgdown ≥ 0 ∀g
z_lower_g = M.variable('z_lower_g', nsupply, binary_domain)
M.constraint(Expr.sub(pG,
                      Expr.mul(bigM, z_lower_g)), Domain.lessThan(0.))
not_z_lower_g = Expr.sub(1., z_lower_g)
M.constraint(Expr.sub(muGdown,
                      Expr.mul(bigM, not_z_lower_g)), Domain.lessThan(0.))

# 0 ≤ −Bn,m (θn − θm ) + Fn,m ⊥ ηupn,m ≥ 0
z_upper_eta = M.variable('z_upper_eta', n_bi_lines, binary_domain)
M.constraint(Expr.sub(Expr.sub(bi_lines['capacity (MW)'].values, line_flows),
                      Expr.mul(bigM, z_upper_eta)), Domain.lessThan(0.))
not_z_upper_eta = Expr.sub(1., z_upper_eta)
M.constraint(Expr.sub(etaup,
                      Expr.mul(bigM, not_z_upper_eta)), Domain.lessThan(0.))

# 0 ≤ Bn,m (θn − θm ) + Fn,m ⊥ ηlown,m ≥ 0
z_lower_eta = M.variable('z_lower_eta', n_bi_lines, binary_domain)
M.constraint(Expr.sub(Expr.add(bi_lines['capacity (MW)'].values, line_flows),
                      Expr.mul(bigM, z_lower_eta)), Domain.lessThan(0.))
not_z_lower_eta = Expr.sub(1., z_lower_eta)
M.constraint(Expr.sub(etadown,
                      Expr.mul(bigM, not_z_lower_eta)), Domain.lessThan(0.))



## dummy objective constraint
M.objective('obj', ObjectiveSense.Maximize, M.variable(Domain.equalsTo(0.)))
M.solve()

if M.getPrimalSolutionStatus() != SolutionStatus.Optimal:
    M.writeTask('proj1kkt.opf')

demand['consumed (MW)'] = pD.level()
supply['supplied (MW)'] = pG.level()
bi_lines['flow (MW)'] = line_flows.level()
buses = pd.DataFrame({
    'volt. angle (rad)': theta.level(),
    'node price ($/MW)': lambdaN.level()
}, index=nodes)

In [10]:
summary = pd.concat([
    buses['node price ($/MW)'],
    demand.groupby('node')['consumed (MW)'].sum(),
    supply.groupby('node')['supplied (MW)'].sum(),
], axis=1)
summary.columns = ['price ($/MW)', 'consumed (MW)', 'supplied (MW)']
display(summary)
print(summary.to_latex())

,price ($/MW),consumed (MW),supplied (MW)
North,-50.0,12854.2,12854.2
Central,-50.0,28181.2,28181.2
South,-50.0,16425.5,16425.5


\begin{tabular}{lrrr}
\toprule
{} &  price (\$/MW) &  consumed (MW) &  supplied (MW) \\
\midrule
North   &         -50.0 &        12854.2 &        12854.2 \\
Central &         -50.0 &        28181.2 &        28181.2 \\
South   &         -50.0 &        16425.5 &        16425.5 \\
\bottomrule
\end{tabular}



In [11]:
print('Energy scheduled: {:g}'.format(demand['consumed (MW)'].sum()))
print('Objective value: {:g}'.format(M.primalObjValue()))
display(bi_lines)
display(buses)
display(demand.sort_values('bid ($/MW)'))
display(supply.sort_values('offer ($/MW)'))

Energy scheduled: 57460.9
Objective value: 0


source     dest  capacity (MW)  susceptance (S)  flow (MW)
source  dest                                                                
North   Central    North  Central         5000.0         100000.0        0.0
        South      North    South         5000.0         100000.0        0.0
Central South    Central    South         5000.0         100000.0        0.0
        North    Central    North         5000.0         100000.0        0.0
South   North      South    North         5000.0         100000.0        0.0
        Central    South  Central         5000.0         100000.0        0.0

,volt. angle (rad),node price ($/MW)
North,0.0,-50.0
Central,0.0,-50.0
South,0.0,-50.0


,node,demand (MW),bid ($/MW),consumed (MW)
id,,,,
1656071284,South,58.0,4.21,58.0
1656072147_0,South,27.0,11.33,27.0
122073523_7,Central,3.1,11.47,3.1
122073523_6,Central,3.1,11.99,3.1
122073523_5,Central,3.1,12.51,3.1
...,...,...,...,...
816860633,Central,5.0,2000.00,5.0
1601140898,Central,3.0,2000.00,3.0
1066937520,Central,30.0,2000.00,30.0


,node,capacity (MW),offer ($/MW),supplied (MW)
id,,,,
A14605_0,North,1.0,-50.0,1.0
A6994_0,North,1.0,-50.0,1.0
A7548_0,North,1.0,-50.0,1.0
A6027_0,North,1.0,-50.0,1.0
A6999_0,North,1.0,-50.0,1.0
...,...,...,...,...
A14896_1,North,69.0,999.0,69.0
A14517_1,Central,19.0,999.0,19.0
A12762_1,North,23.0,999.0,23.0


In [12]:
print(log.getvalue())

Problem
  Name                   : kkt             
  Objective sense        : max             
  Type                   : LO (linear optimization problem)
  Constraints            : 19666           
  Cones                  : 0               
  Scalar variables       : 19669           
  Matrix variables       : 0               
  Integer variables      : 7864            

Optimizer started.
Mixed integer optimizer started.
Threads used: 6
Presolve started.
Presolve terminated. Time = 1.18
Presolved problem: 7265 variables, 7267 constraints, 18161 non-zeros
Presolved problem: 0 general integer, 3631 binary, 3634 continuous
Clique table size: 276
BRANCHES RELAXS   ACT_NDS  DEPTH    BEST_INT_OBJ         BEST_RELAX_OBJ       REL_GAP(%)  TIME  
0        1        1        0        NA                   -0.0000000000e+00    NA          1.4   
Cut generation started.
0        1        1        0        NA                   -0.0000000000e+00    NA          4.6   
Cut generation terminated. Tim